# Trelegemeproblemet

Trelegemeproblemet handler om å simulere planetbanene til tre legemer som interagerer med en sentralkraft. I dette tilfellet skal vi se på tilfellet der sentralkraften er gravitasjonskraften. Du skal skrive en kode som regner ut banene til

1. En planet med en masse tilsvarende jorda.
2. En stjerne med masse tilsvarende sola.
3. En stjerne med masse som er fire ganger så stor som sola.

````{tab} 2d-simulering
```{figure} ../../codes/physics/solar_system/animations/three_body_system_2d.gif
:name: three_body_system_2d

Animasjon av trelegemeproblemet i 2D over 10 jordår.
```
````
````{tab} 3d-simulering
```{figure} ../../codes/physics/solar_system/animations/three_body_system_3d.gif
:name: three_body_system_3d

Animasjon av trelegemeproblemet i 3D over 10 jordår.
```
````

## Bakgrunnsteori

### Gravitasjonskraften fra ett legeme på et annet

I fysikken i videregående skole lærer man at gravitasjonsloven kan skrives som

$$
F = -\gamma \frac{m_1 m_2}{r^2},
$$

der $r$ er avstanden mellom de to legemene, $m_1$ og $m_2$ er massene til objektene og $\gamma$ er gravitasjonskonstanten.

Svakheten med denne loven er at kraften er en vektorstørrelse og har retning, mens vi med loven over kun regner ut størrelsen. I tillegg antar den at kilden til gravitasjonskraften er i origo i koordinatsystemet vi bruker. I stedet kan vi skrive loven slik:

$$
\vec{F}_1 = -\gamma m_1 m_2 \frac{\vec{r}_1 - \vec{r}_2}{|\vec{r}_1 - \vec{r}_2|^3},
$$

der kraften virker på masse $m_1$ i posisjon $\vec{r}_1$ fra legeme med masse $m_2$ i posisjon $\vec{r}_2$. Tanken her er at legeme 2 er solen, og legeme 1 er alle de andre planetene. Fordelen med denne formuleringen er at vi får med retningsvektoren også! 

### Gravitasjonskraften på ett legeme fra $n - 1$ andre legemer

Når vi skal simulere et system bestående av flere legemer, vil det virke en gravitasjonskraft fra hvert legeme på hvert annet i systemet. Hvis det er tre legemer, vil ett legeme påvirkes av gravitasjonskraften fra de to andre, og vice versa. Vi trenger en utvidet versjon av gravitasjonsloven når det er flere legemer til stedet. Kraften på legeme $i$ er da gitt ved

$$
\vec{F}_i = -\sum_{j\neq i} \gamma m_i m_j \frac{\vec{r}_i - \vec{r}_j}{|\vec{r}_i - \vec{r}_j|^3},
$$

der $\vec{r}_i$ er posisjonen til legemet kraften virker på, $m_i$ er massen til legeme kraften virker på, $m_j$ er kraften til legeme $j$ og $\vec{r}_j$ er posisjonen til legeme $j$. Vi summerer over alle legemer bortsett fra legeme $i$ selv, som er grunnen til at det står $\sum_{j\neq i}$. Vi kommer tilbake til hvordan vi håndterer det med kode senere.

### Newtons 2.lov

Når vi ønsker å finne bevegelsen til et legeme over tid, er Newtons 2.lov vår *go-to* lov. Kjenner vi alle kreftene på legeme, kan vi finne bevegelsen til legemet over tid. Newtons 2.lov kan oppsummeres som


$$
\sum_{n} \vec{F}_n = m_i\vec{a}_i,
$$

I dette tilfellet, er summen av kreftene den totale gravitasjonskraften som virker på legemet. For legemet $i$ kan vi skrive dette som

$$
m_i\vec{a}_i = -\gamma \sum_{j \neq i} m_i m_j \frac{\vec{r}_i - \vec{r}_j}{|\vec{r}_i - \vec{r}_j|^3}, 
$$

som gir

$$
\vec{a}_i = -\gamma \sum_{j \neq i} m_j \frac{\vec{r}_i - \vec{r}_j}{|\vec{r}_i - \vec{r}_j|^3}.
$$


```{dropdown} Eksempel på tre legemer
Tenk deg at vi har 3 legemer, og vi ønsker å regne ut akselerasjonen til legeme 2. Da har vi 
- Tre posisjoner $\vec{r}_1$, $\vec{r}_2$ og $\vec{r}_3$.
- Tre masser $m_1$, $m_2$ og $m_3$.

Men vi må ekskludere legeme 2 fra summen, siden vi ikke ønsker å regne ut gravitasjonskraften fra legeme 2 på seg selv (noe som ikke gir mening når vi betrakter det som et punktlegeme).

Plugger vi det inn i akselerasjonsformelen, blir det da slik:

$$
\vec{a}_2 = -\gamma \left( m_1 \frac{\vec{r}_2 - \vec{r}_1}{|\vec{r}_2 - \vec{r}_1|^3} + m_3 \frac{\vec{r}_2 - \vec{r}_3}{|\vec{r}_2 - \vec{r}_3|^3} \right).
$$


*Merknad: Dette er bare ment for å opplyse deg om hva formelen med $\sum_{j \neq i}$ betyr for noe. Når du skal kode dette, handler det hele bare å sjekke om `i != j` i en `if`-test, for å så regne ut summen som normalt.*

```

### Velocity Verlet-algoritmen

Solsystemet er et system der energien er bevart. Du har kanskje sett Euler-Comer før, men her skal vi bruke en annen numerisk metode som også bevarer energi, en såkalt *symplektisk* integrator. En slik metode vil holde energien konstant i gjennomsnitt over tid. I praksis vil vi observere at verdien til energien svinger frem og tilbake om startverdien. Euler-Cromer svinger i mye større grad enn den metoden vi skal bruke.

Vi skal benytte oss av en annen symplektisk metode som kallet for *velocity Verlet*-algoritmen. Anta at

$$
\vec{a}_i(t) = \vec{f}(\vec{r}_1(t), \vec{r}_2(t), \ldots, \vec{r}_{n}(t)),
$$

gir oss akselerasjon ved tidspunkt $t$, og $\vec{r}_1(t), \vec{r}_2(t), \ldots, \vec{r}_n(t)$ er posisjonene til planetene ved tidspunkt $t$. Da kan vi regne ut neste posisjon og hastighet til planet $i$ (med en steglengde $h$ tid):

$$
\begin{align*}
\vec{r}_i(t + h) & \approx \vec{r}_i(t) + \vec{r}_i(t) h + \frac{1}{2}\vec{a}_i(t) h^2, \\
\vec{v}_i(t + h) & \approx \vec{v}_i(t) + \frac{1}{2}\left[\vec{a}_i(t) + \vec{a}_i(t + h)\right]h
\end{align*}
$$

```{admonition} Typisk fallgruve
:class: tip, dropdown

En typisk fallgruve her er at man bruke algoritmen direkte på én planet av gangen, men siden akselerasjonen til hver planet er avhengig av posisjonen til alle de andre, må bare gjøre *steg 1* av algoritmen på **alle** planetene før man gjøre steg to på **alle** planetene. Med andre ord

1. Oppdater posisjonen til **alle** planetene først (inkludert solen). 
2. Deretter oppdater hastigheten til **alle** planetene (inkludert solen). 
```

### Astronomiske enheter

Når man regner på planetbaner, er det nyttig å jobbe med andre enheter fordi størrelsene vi jobber med er såpass store. Avstandene spenner flere millioner kilometer og tiden spenner flere jordår. Derfor velger vi å:

- Måle avstander i astronomiske enheter, AU. (1 AU er gjennomsnittsavstanden mellom jorden og solen. Ca. 150 millioner km.)
- Mål hastigheter i AU/år (AU/jordår). 
- Måle tid i år (jordår). 
- Masse måles i antall *solmasser* $M_\odot = 2\cdot 10^{30} \ \text{kg}$. (1 solmasse er massen til sola vår). *Hint: for å omgjøre en masse til solmasser, deler man massen med solmassen. Da får du hvor mange solmasser den massen utgjør*.
- Gravitasjonskonstanten $\gamma$ er tilnærmet gitt ved $\gamma = 4\pi^2 \ \text{AU}^3/(\text{år}^2 M_\odot)$ i astronomiske enheter. Det er en konsekvens som følger fra Keplers 3.lov. Hvis du tenker over enhetene, så gir de mening fordi vi har satt $\text{m} \to \text{AU}$, og $\text{s} \to \text{år}$, og $\text{kg} \to M_\odot$.

### Initialbetingelser

Her skal vi liste opp initialbetingelsene til legemene inkludert massene deres. Disse trenger du for å kjøre simuleringen.


**Tabell 1: Initialbetingelser for solsystemet.**
| Legeme | Masse $m$ [kg] | $x$ [AU] | $y$ [AU] | $z$ [AU] | $v_x$ [km/s] | $v_y$ [km/s] | $v_z$ [km/s] |
|--------|----------------|----------|----------|----------|---------------|---------------|--------------|
| Planet  | $6 \cdot 10^{24}$ | -1.5 | 0 | 0 | 0 | -1 | 0 |
| Lett stjerne  | $2 \cdot 10^{30}$ | 0 | 0 | 0 | 0 | 30 | 0 |
| Tung stjerne  | $8 \cdot 10^{30}$ | 3 | 0 | 0 | 0 | -7.5 | 0 |


*Merk at du må konvertere hastighetene fra km/s til AU/år!*

## Oppgaver

### Oppgave 1: lag arrays med initialbetingelser

I denne oppgaven skal du gjøre forarbeidet for simuleringen. Du skal lage en funksjon som

- Lage et numpy array `r` med initialposisjonene til legemene. Dette skal ha `shape=(antall_legemer, 3)`.
- Lage et numpy array `v` med intialhastighetene til legemene. Dette skal ha `shape=(antall_legemer, 3)`.
- Lage et numpy array `m` med massene til legemene. Dette skal ha `shape=(antall_legemer,)`.
- Gir navn til alle legemene.

Funksjonen skal reuturnere `r`, `v`, `m`, `navn`.

*Du kan ta utgangspunkt i kodeskallet under. Du må fylle inn der det står `NotImplemented`*.

````{dropdown} Kodehint:

For å få laget arrayet med riktig shape her kan du skrive lage et array fra en liste. For eksempel er en liste med `shape` tilsvarende `(3, 2)` (tilsvarer 3 lister med lengde 2 inni en liste) gitt ved en *nøstet liste* på formen

```python
import numpy as np

# Lager liste med form (3, 2)
min_liste = [
    [1, 2], 
    [3, 4], 
    [5, 6]
]

# Lager arrayet fra listen
arr = np.array(min_liste)
```
````

In [ ]:
import numpy as np

def get_init_data():
    # Sett navn på legemene. 
    navn = ["planet", "lett stjerne", "tung stjerne"]

    # Lage massevektoren
    solmasse = 2e30
    jordmasse = 6e24
    m = [jordmasse, solmasse, 4 * solmasse]
    m = np.array(m) # omdanner til array
    m /= solmasse # konverterer alt målt i solmasser.

    # Lage posisjonsvektoren
    r = [
        [-1.5, 0, 0], # initialposisjon til planeten i AU
        [NotImplemented], # initialposisjon til lett stjerne i AU
        [NotImplemented], # initialposisjon til tung stjerne i AU
    ]
    r = np.array(r) # omdanner til array

    # Lage fartsvektoren
    v = [
        [NotImplemented], # initialfart til planeten i AU/yr
        [NotImplemented], # initialfart til lett stjerne i AU/yr
        [NotImplemented], # initialfart til tung stjerne i AU/yr
    ]

    v = NotImplemented # omdann til array


    # Konvertere hastigheter fra km/s til AU/år
    # Gjør det gjerne i to steg som foreslått under
    v = NotImplemented # konverterer fra km/s til AU/s
    v = NotImplemented # konverter fra AU/s til AU/år

    return r, v, m, navn


# Eksempel på funksjonskall for å hente ut dataene
r, v, m, navn = get_init_data() 


````{dropdown} Løsningsforslag

Vi må fylle inn det som mangler i kodeskallet. Vi kan gjøre det slik:

```python
import numpy as np
def get_init_data():
    navn = ["planet", "lett stjerne", "tung stjerne"]
    # Lage massevektoren
    solmasse = 2e30
    jordmasse = 6e24
    m = [jordmasse, solmasse, 4 * solmasse]
    m = np.array(m) # omdanner til array
    m /= solmasse # konverterer alt målt i solmasser.

    # Lage posisjonsvektoren
    r = [
        [-1.5, 0, 0], # initialposisjon til planeten i AU
        [0, 0, 0], # initialposisjon til lett stjerne i AU
        [3, 0, 0], # initialposisjon til tung stjerne i AU
    ]
    r = np.array(r) # omdanner til array

    # Lage fartsvektoren
    v = [
        [0, -1, 0], # initialfart til planeten i AU/yr
        [0, 30, 0], # initialfart til lett stjerne i AU/yr
        [0, -7.5, 0], # initialfart til tung stjerne i AU/yr
    ]

    v = np.array(v) # omdann til array


    # Konvertere hastigheter fra km/s til AU/år
    # Gjør det gjerne i to steg som foreslått under
    v /= 150e6 # konverterer fra km/s til AU/s
    v *= 365.25 * 24 * 60 * 60 # konverter fra AU/s til AU/år

    return r, v, m, navn

```

I selve konverteringen har vi brukt at det er $1 \ \text{AU} = 150\cdot 10^6 \ \text{km}$ og at $1 år = 365.25 \cdot 24 \cdot 60 \cdot 60 \ \text{s}$.


````

### Oppgave 2: Lag en funksjon som regner ut akselerasjonen

For å regne ut planetbanene, må vi først og fremst være i stand til å hente ut hvilken akselerasjon hvert legeme opplever på et gitt tidspunkt. Dette gjør vi ved å lage en funksjon `akselerasjon(r, m, i)` som tar inn posisjonsvektoren `r`, massene `m`, og hvilket legeme `i` vi skal regne ut akselerasjonen til, og returnerer akselerasjonsvektoren `a`. 

*Du kan ta utgangspunkt i kodeskallet under. Du må fylle inn der det står `NotImplemented`*

In [3]:
import numpy as np

def get_akselerasjon(r, m, i, G=4 * np.pi**2):
    a = np.zeros(3) # tre-dimensjonal vektor
    G = 4 * np.pi**2 # gravitasjonskonstanten
    for j in range(r.shape[0]):
        if i != j:
            dr = r[i] - r[j]
            dr_norm = np.linalg.norm(dr)
            a -= m[j] * dr / dr_norm**3

    a *= G
    return a


````{dropdown} Løsningsforslag

```python 
import numpy as np

def get_akselerasjon(r, m, i, G=4 * np.pi**2):
    a = np.zeros(3) # tre-dimensjonal vektor
    G = 4 * np.pi**2 # gravitasjonskonstanten
    for j in range(r.shape[0]):
        if i != j:
            dr = r[i] - r[j]
            dr_norm = np.linalg.norm(dr)
            a -= m[j] * dr / dr_norm**3

    a *= G
    return a
```


````

### Oppgave 3: Lag en funksjon som regner ut neste posisjon og hastighet

Her skal du skrive en funksjon `verlet_step` som tar inn:

- posisjonene til legemene `r`.
- hastighetene til legemene `v`.
- massene til legemene `m`.
- tidssteg `dt`
- akselerasjonsfunksjonen `get_akselerasjon`

Funksjonen skal bruke velocity Verlet-algoritmen til å regne ut og returnere
- Neste posisjon for alle legemene. 
- Neste hastighetet for alle legemene.


*Du kan ta utgangspunkt i kodeskallet under. Du må fylle inn der det står `NotImplemented`*.

In [ ]:
def verlet_step(r, v, m, dt, get_akselerasjon):
    a_gammel = np.zeros(shape=r.shape)
    for i in range(len(r)):
        a_gammel[i] = get_akselerasjon(r=r, m=m, i=i)

    r = NotImplemented

    a_ny = NotImplemented
    for i in range(len(r)):
        a_ny[i] = NotImplemented

    v = NotImplemented

    return r, v

In [ ]:
def verlet_step(r, v, m, dt, get_akselerasjon):
    a_gammel = np.zeros(shape=r.shape)
    for i in range(len(r)):
        a_gammel[i] = get_akselerasjon(r=r, m=m, i=i)
    
    r = r + v * dt + 0.5 * a_gammel * dt**2

    a_ny = np.zeros(shape=r.shape)
    for i in range(len(r)):
        a_ny[i] = get_akselerasjon(r=r, m=m, i=i)
    
    v = v + 0.5 * (a_gammel + a_ny) * dt

    return r, v

### Oppgave 4: Sett det hele sammen til å lage en simulering

Her skal du skrive en `main`-funksjon der du setter det hele sammen. Det er viktig at `main`-funksjonen defineres på bunnen så alle de andre funksjonene er tilgjengelig.

*Du kan ta utgangspunkt i kodeskallet under. Du må fylle inn der det står `NotImplemented`*.

````{dropdown} Hjelpefunksjon for å lage animasjon i 2d
Får å se hvordan man bruker animasjonsfunksjonen, se i **kodeskallet** under. 
Bare kopier denne koden på toppen av koden din og kjør `main`-funksjonen når du har skrevet ferdig den.


```python
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.colors import to_rgba_array

def create_animation_2d(positions, names, tail_length=10, interval=50):
    T, N, _ = positions.shape

    fig, ax = plt.subplots(figsize=(8, 5), facecolor='black')
    ax.set_facecolor('black')
    ax.grid(False)
    
    xlim = np.max(np.abs(positions[:, :, 0]))
    ylim = np.max(np.abs(positions[:, :, 1]))
    ax.set_xlim([-xlim, xlim])
    ax.set_ylim([-ylim, ylim])

    # Colors
    colors = to_rgba_array(plt.cm.gist_rainbow(np.linspace(0, 1, N)))  # Use gist_rainbow colormap for neon colors

    lines = [ax.plot([], [], '-', lw=2, alpha=0.6)[0] for _ in range(N)]
    dots = [ax.plot([], [], 'o', markersize=15, color=colors[i])[0] for i in range(N)]  # Use corresponding colors for dots and larger markersize

    # Initialize tail data
    tail_data = [np.empty((0, 2)) for _ in range(N)]

    def init():
        for line, dot in zip(lines, dots):
            line.set_data([], [])
            dot.set_data([], [])
        return lines + dots

    def update(frame):
        for i, (line, dot) in enumerate(zip(lines, dots)):
            tail_data[i] = np.vstack((tail_data[i], positions[frame, i]))

            # Update line data with tail
            current_length = len(tail_data[i])
            line_data = tail_data[i][-tail_length:] if current_length > tail_length else tail_data[i]

            # Set line data
            line.set_data(line_data[:, 0], line_data[:, 1])
            line.set_color(colors[i])

            # Update dot data with current position
            x, y = positions[frame, i]
            dot.set_data(x, y)
            dot.set_color(colors[i])  # Set color to match object
            dot.set_zorder(3)  # Make sure dots appear on top

            # Reset tail data when frame is 0
            if frame == 0:
                tail_data[i] = np.empty((0, 2))

        return lines + dots

    # Add legend
    ax.legend(dots, names, loc='upper right', facecolor='black', fontsize=10, framealpha=0.8)
    plt.setp(plt.gca().get_legend().get_texts(), color='w')  # Set legend text color to white

    animation = FuncAnimation(fig, update, frames=T, init_func=init, interval=interval, blit=False)

    # plt.show()

    return animation
```


````

````{dropdown} Hjelpefunksjon for å lage animasjon i 3d
Får å se hvordan man bruker animasjonsfunksjonen, se i **kodeskallet** under. 
Bare kopier denne koden på toppen av koden din og kjør `main`-funksjonen når du har skrevet ferdig den.


```python
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import to_rgba_array

def create_animation_3d(positions, names, tail_length=10, interval=50):
    T, N, _ = positions.shape

    fig = plt.figure(figsize=(8, 5), facecolor='black')
    ax = fig.add_subplot(111, projection='3d')
    ax.grid(False)
    ax.set_axis_off()

    xlim = np.max(np.abs(positions[:, :, 0]))
    ylim = np.max(np.abs(positions[:, :, 1]))
    zlim = np.max(np.abs(positions[:, :, 2]))
    ax.set_xlim([-xlim, xlim])
    ax.set_ylim([-ylim, ylim])
    ax.set_zlim([-zlim, zlim])

    # Colors
    colors = to_rgba_array(plt.cm.gist_rainbow(np.linspace(0, 1, N)))  # Use gist_rainbow colormap for neon colors

    lines = [ax.plot([], [], [], '-', lw=2, alpha=0.6)[0] for _ in range(N)]
    dots = [ax.plot([], [], [], 'o', markersize=10, color=colors[i])[0] for i in range(N)]  # Use corresponding colors for dots

    # Initialize tail data
    tail_data = [np.empty((0, 3)) for _ in range(N)]

    def init():
        for line, dot in zip(lines, dots):
            line.set_data([], [])
            line.set_3d_properties([])
            dot.set_data([], [])
            dot.set_3d_properties([])
        return lines + dots

    def update(frame):
        # Rotate the perspective
        ax.view_init(30, 0.3 * frame)

        for i, (line, dot) in enumerate(zip(lines, dots)):
            tail_data[i] = np.vstack((tail_data[i], positions[frame, i]))

            # Update line data with tail
            line_data = tail_data[i][-tail_length:] # Now using constant tail length

            # Set line data
            line.set_data(line_data[:, 0], line_data[:, 1])
            line.set_3d_properties(line_data[:, 2])
            line.set_color(colors[i])

            # Update dot data with current position
            x, y, z = positions[frame, i]
            dot.set_data(x, y)
            dot.set_3d_properties(z)
            dot.set_color(colors[i])  # Set color to match object
            dot.set_zorder(3)  # Bring dots to front

        return lines + dots

    # Add legend
    ax.legend(dots, names, loc='upper right', facecolor='black', fontsize=10, framealpha=0.8)
    plt.setp(plt.gca().get_legend().get_texts(), color='w')  # Set legend text color to white

    animation = FuncAnimation(fig, update, frames=T, init_func=init, interval=interval, blit=False, repeat=True, repeat_delay=8)

    # plt.show()

    return animation
```


````

In [ ]:
from tqdm import trange # gir progresjonsbar. Erstatter `range` i `for`-løkker
def main():
    n_tidssteg = 1_000_000 # 1 million tidssteg
    dt = 1e-5 # 0.00001 år per tidssteg

    r, v, m, navn = NotImplemented # hent initialverdier
    

    posisjoner = np.zeros(shape=(n_tidssteg, len(r), 3))
    hastigheter = np.zeros(shape=(n_tidssteg, len(r), 3))

    posisjoner[0, ...] = r[:]
    hastigheter[0, ...] = v[:]

    for i in trange(1, n_tidssteg):
        
        # Regn ut neste posisjoner og hastigheter med velocity Verlet
        r, v = NotImplemented

        # Lagre posisjonene og hastighetene
        posisjoner[i, ...] = NotImplemented
        hastigheter[i, ...] = NotImplemented


    # Lager 2d-animasjon og lagrer den som gif
    ani_2d = create_animation_2d(posisjoner, navn)
    ani_2d.save("trelegemeproblem_2d.gif", fps=60)

    # Lager 3d-animasjon og lagrer den som gif
    ani_3d = create_animation_3d(posisjoner, navn)
    ani_3d.save("trelegemeproblem_3d.gif", fps=60)

````{dropdown} Løsningsforslag

```python
def main():
    n_tidssteg = 1_000_000 # 1 million tidssteg
    dt = 1e-5 # 0.00001 år per tidssteg

    r, v, m, navn = get_init_data() # hent initialverdier
    

    posisjoner = np.zeros(shape=(n_tidssteg, len(r), 3))
    hastigheter = np.zeros(shape=(n_tidssteg, len(r), 3))

    posisjoner[0, ...] = r[:]
    hastigheter[0, ...] = v[:]

    for i in trange(1, n_tidssteg):
        
        # Regn ut neste posisjoner og hastigheter med velocity Verlet
        r, v = verlet_step(
            r=r,
            v=v,
            m=m,
            dt=dt,
            get_akselerasjon=get_akselerasjon,
        )

        # Lagre posisjonene og hastighetene
        posisjoner[i, ...] = r[:]
        hastigheter[i, ...] = v[:]


    # Lager 2d-animasjon og lagrer den som gif
    ani_2d = create_animation_2d(posisjoner, navn)
    ani_2d.save("trelegemeproblem_2d.gif", fps=60)

    # Lager 3d-animasjon og lagrer den som gif
    ani_3d = create_animation_3d(posisjoner, navn)
    ani_3d.save("trelegemeproblem_3d.gif", fps=60)
```

````